# PosgreSQL Connection

### Libraries Used

In [1]:
import psycopg2
import pandas as pd

### Connection Parameters

In [2]:
HOST = "localhost"
DATABASE = "postgres"
USER = "postgres"
PASSWORD = "rishabh"

### Connect to DataBase

In [3]:
# connect() ---> creates a new database session and returns a new instance of the connection class
conn = psycopg2.connect(host=HOST,database=DATABASE, user=USER, password=PASSWORD)

# Open a cursor to perform database operations
db_cursor = conn.cursor()

# Close communication with the database
# db_cursor.close()

### Get the list of Tables

In [4]:
# # Execute a command: this creates a new table
# db_cursor.execute(
#     "SELECT table_schema,table_name FROM information_schema.tables WHERE table_schema = 'public';")

# # retrieve data from the database 
# list_tables = db_cursor.fetchall()
# list_tables

### Get the Table

In [5]:
# table = db_cursor.execute("select * from actor")
table = "select * from staff" 

# Read SQL query or database table into a DataFrame
df = pd.read_sql(table,conn)

In [13]:
df1 = pd.read_excel("/home/bluepi/Documents/dashboard_city_view.xlsx", index_col=0)

### Rename the columns

In [16]:
renamed_columns = {"order_date": "Order Date",
                   "city": "City", 
                   "article": "MAT", 
                   "qoh": "SIH",
                   "last_year_same_day_sale": "LY-Sales",
                   "last_week_same_day_sale": "LW-Sales", 
                   "d2_sit": "D-2-SIT", 
                   "d1_sit": "D-1-SIT",
                   "forecast": "Sales Forecast", 
                   "actual_ordering_qty": "Sys Suggested Order Qty", 
                   "modified_ordering_qty": "Store Order Qty", 
                   "sourcer_modified_ordering_qty": "HO Order Qty", 
                   "final_order_qty ": "Final Order Qty"
                 }

df1 = df1.rename(columns=renamed_columns)
header_list = list(df1.columns)
header_list.extend(["MDESC","CAT","SUB CAT","MTD Dump%","Order Variation"])
df1 = df1.reindex(columns=header_list)


In [17]:
df1

,City,MAT,Order Date,sales_prev,Sys Suggested Order Qty,Store Order Qty,dump_qty,Sales Forecast,LY-Sales,LW-Sales,...,MDESC,CAT,SUB CAT,MTD Dump%,Order Variation,MDESC,CAT,SUB CAT,MTD Dump%,Order Variation
0,AHM,1008699,2020-04-11,215.040,174.768,NaN,-18.256001,196.0,NaN,134.496,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AHM,1008699,2020-04-12,241.052,174.016,NaN,-12.780000,236.0,NaN,170.440,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AHM,1008699,2020-04-13,183.716,124.084,NaN,-8.000000,162.0,NaN,162.776,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AHM,1008699,2020-04-14,182.196,54.000,NaN,-15.280000,146.0,NaN,187.048,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AHM,1008699,2020-04-15,224.452,61.888,NaN,-4.800000,158.0,NaN,228.480,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10615,SUR,1336054,2020-04-16,0.000,0.000,NaN,NaN,0.0,NaN,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10616,SUR,1336054,2020-04-17,0.000,0.000,NaN,NaN,0.0,NaN,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10617,SUR,1336054,2020-04-18,NaN,0.000,NaN,NaN,0.0,NaN,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10618,SUR,1336054,2020-04-19,NaN,0.000,NaN,NaN,0.0,NaN,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Write it to EXCEL

In [8]:
filename ='staff.xlsx' 
df.to_excel(filename,header=True)

# Mail Connection

## Setting up the Mail Address 

> Instead of turning ON the **Less Secure App Access** option in your google account. 
> We will follow this procedure.

1. First, you need to enable **2-Step Verification** ( Goto **Manage Your Google Account > Security** ) 
<img src="files/w1.png" width="600" />
2. Next, create an app password. Just select **App passwords**
    <img src="files/w2.png" width="600" />
3. Type the name(any)
4. Generate the password
5. Use this password inplace of mail password
6. Done.





### Libraries Used

In [9]:
import email, smtplib, ssl

from email import encoders

# This is the base class for all the MIME-specific subclasses of Message
from email.mime.base import MIMEBase

# A subclass of MIMEBase
# It is an intermediate base class for MIME messages that are multipart
from email.mime.multipart import MIMEMultipart

# MIMEText is used to create MIME objects of major_type : text
from email.mime.text import MIMEText

port = 465  # For SSL

### Connection Parameters

In [9]:
sender_email = "test.mail.bluepi@gmail.com"
receiver_email = "rishabh@bluepi.in"

subject = "An email with attachment from Python"
body = "This is an email with attachment sent from Python"

password = "wfsgwvsegjnukxzt"
# input("Type your password and press enter: ")

### Send the Email

In [10]:
# Create a multipart message and set headers
message = MIMEMultipart()
message["From"] = sender_email
message["To"] = receiver_email
message["Subject"] = subject
# message["Bcc"] = receiver_email

# Add body to email
message.attach(MIMEText(body, "plain"))

# filename = list_tables[0][1] + '.xlsx'  # In same directory as script

# Open CSV file in binary mode
with open(filename, "rb") as attachment:
# Add file as application/octet-stream
# Email client can usually download this automatically as attachment
    part = MIMEBase("application", "octet-stream")
    part.set_payload(attachment.read())

# Encode file in ASCII characters to send by email    
encoders.encode_base64(part)

# Add header as key/value pair to attachment part
part.add_header(
    "Content-Disposition",
    f"attachment; filename= {filename}",
)

# Add attachment to message and convert message to string
message.attach(part)
text = message.as_string()

# Create a secure SSL context
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, text)


NameError: name 'MIMEMultipart' is not defined